# Setup

## Google Only

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('/content/drive/My Drive/Code/autocomplete_me')

In [3]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 20
-rw------- 1 root root 2516 Jul 12 14:10 predict_utils.py
drwx------ 2 root root 4096 Jul 11 13:20 __pycache__
-rw------- 1 root root 3266 Jul 12 16:08 reader.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [4]:
from src import utils, reader, predict_utils

Using TensorFlow backend.


In [27]:
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)

<module 'src.predict_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py'>

# Build Model

## Training

In [5]:
text = reader.read_bbc_politics()
content_type = 'BBC-pol'

In [6]:
text[0]

'Tory expert denies defeat warning\n\nThe Conservatives\' campaign director has denied a report claiming he warned Michael Howard the party could not win the next general election.\n\nThe Times on Monday said Australian Lynton Crosby told the party leader to focus on trying to increase the Tories\' Commons presence by 25 to 30 seats. But Mr Crosby said in a statement: "I have never had any such conversation... and I do not hold that view." Mr Howard later added there was not "one iota" of truth in the report. The strategist helped Australia\'s PM, John Howard, win four elections. Mr Howard appointed Mr Crosby as his elections chief last October. Mr Crosby\'s statement said: "The Conservative Party has been making an impact on the issues of lower tax and controlled immigration over the past week." It added: "The Labour Party will be wanting to do all they can to distract attention away from the issues that really matter to people."\n'

### Long Text

In [66]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text)

There are 14136 unique words.
There are 183458 sequences.


In [67]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 4852 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:175: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.06223089,  0.03835242,  0.08488411, ..., -0.04284497,
         0.08662399, -0.00527513],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

#### Unidirectional, 1 layer, Trainable

In [11]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [12]:
train_history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-50_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 128420 samples, validate on 55038 samples
Epoch 1/150
128420/128420 [==============================] - 20s 154us/step - loss: 7.7903 - accuracy: 0.0392 - val_loss: 6.9065 - val_accuracy: 0.0537
Epoch 2/150
128420/128420 [==============================] - 18s 137us/step - loss: 6.8572 - accuracy: 0.0509 - val_loss: 6.8394 - val_accuracy: 0.0537
Epoch 3/150
128420/128420 [==============================] - 18s 139us/step - loss: 6.7501 - accuracy: 0.0516 - val_loss: 6.8059 - val_accuracy: 0.0537
Epoch 4/150
128420/128420 [==============================] - 18s 137us/step - loss: 6.6999 - accuracy: 0.0531 - val_loss: 6.8094 - val_accuracy: 0.0646
Epoch 5/150
128420/128420 [==============================] - 18s 137us/step - loss: 6.6622 - accuracy: 0.0614 - val_loss: 6.7902 - val_accuracy: 0.0692
Epoch 6/150
128420/128420 [==============================] - 18s 137us/step - loss: 6.5839 - accuracy: 0.0724 - val_loss: 6.7004 - val_accuracy: 0.0750
Epoch 7/150
128420/128420 [==========

UnboundLocalError: ignored

In [68]:
train_history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-50_seq',
    use_pretrained_model=True,
    epochs=500
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 128420 samples, validate on 55038 samples
Epoch 1/500
128420/128420 [==============================] - 18s 141us/step - loss: 5.3775 - accuracy: 0.1558 - val_loss: 6.1201 - val_accuracy: 0.1485
Epoch 2/500
128420/128420 [==============================] - 17s 135us/step - loss: 5.3324 - accuracy: 0.1600 - val_loss: 6.1204 - val_accuracy: 0.1501
Epoch 3/500
128420/128420 [==============================] - 17s 133us/step - loss: 5.2882 - accuracy: 0.1620 - val_loss: 6.1334 - val_accuracy: 0.1513
Epoch 4/500
128420/128420 [==============================] - 17s 134us/step - loss: 5.2429 - accuracy: 0.1632 - val_loss: 6.1377 - val_accuracy: 0.1516
Epoch 5/500
128420/128420 [==============================] - 17s 134us/step - loss: 5.1999 - accuracy: 0.1667 - val_loss: 6.1517 - val_accuracy: 0.1540
Epoch 6/500
128420/128420 [==============================] - 17s 134us/step - loss: 5.1592 - accuracy: 0.1690 - val_loss: 6.1555 - val_accuracy: 0.1550
Epoch 7/500
128420/128420 [==========

#### Unidirectional, 1 layer, trainable, lstm-dropout-0.2, lstm-recurrent-dropout-0.2

In [26]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False,
    lstm_dropout=0.2,
    lstm_recurrent_dropout=0.2,
)

In [27]:
train_history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-50_seq-lstm_drop_0.2-lstm_rec_drop_0.2',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 128420 samples, validate on 55038 samples
Epoch 1/50
128420/128420 [==============================] - 16s 127us/step - loss: 7.7973 - accuracy: 0.0390 - val_loss: 6.9081 - val_accuracy: 0.0537
Epoch 2/50
128420/128420 [==============================] - 16s 125us/step - loss: 6.8475 - accuracy: 0.0506 - val_loss: 6.8325 - val_accuracy: 0.0537
Epoch 3/50
128420/128420 [==============================] - 16s 126us/step - loss: 6.7430 - accuracy: 0.0513 - val_loss: 6.8037 - val_accuracy: 0.0537
Epoch 4/50
128420/128420 [==============================] - 16s 126us/step - loss: 6.6897 - accuracy: 0.0577 - val_loss: 6.7953 - val_accuracy: 0.0696
Epoch 5/50
128420/128420 [==============================] - 16s 125us/step - loss: 6.6323 - accuracy: 0.0674 - val_loss: 6.7517 - val_accuracy: 0.0731
Epoch 6/50
128420/128420 [==============================] - 16s 125us/step - loss: 6.5480 - accuracy: 0.0748 - val_loss: 6.6813 - val_accuracy: 0.0777
Epoch 7/50
128420/128420 [=================

#### Unidirectional, 1 layer, trainable, lstm-dropout-0.5, lstm-recurrent-dropout-0.5

In [23]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False,
    lstm_dropout=0.5,
    lstm_recurrent_dropout=0.5,
)

In [25]:
train_history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-50_seq-lstm_drop_0.5-lstm_rec_drop_0.5',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 128420 samples, validate on 55038 samples
Epoch 1/50
128420/128420 [==============================] - 17s 132us/step - loss: 7.8465 - accuracy: 0.0385 - val_loss: 6.9133 - val_accuracy: 0.0537
Epoch 2/50
128420/128420 [==============================] - 17s 130us/step - loss: 6.8556 - accuracy: 0.0515 - val_loss: 6.8423 - val_accuracy: 0.0537
Epoch 3/50
128420/128420 [==============================] - 17s 132us/step - loss: 6.7569 - accuracy: 0.0524 - val_loss: 6.8120 - val_accuracy: 0.0537
Epoch 4/50
128420/128420 [==============================] - 17s 130us/step - loss: 6.7077 - accuracy: 0.0549 - val_loss: 6.8065 - val_accuracy: 0.0596
Epoch 5/50
128420/128420 [==============================] - 17s 133us/step - loss: 6.6515 - accuracy: 0.0666 - val_loss: 6.7440 - val_accuracy: 0.0725
Epoch 6/50
128420/128420 [==============================] - 17s 131us/step - loss: 6.5521 - accuracy: 0.0742 - val_loss: 6.6689 - val_accuracy: 0.0771
Epoch 7/50
128420/128420 [=================

### Short Text

In [9]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=20)

There are 14136 unique words.
There are 195968 sequences.


In [10]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 4852 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:175: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.06223089,  0.03835242,  0.08488411, ..., -0.04284497,
         0.08662399, -0.00527513],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

#### Unidirectional, 1 layer, Trainable

In [11]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [12]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 137177 samples, validate on 58791 samples
Epoch 1/50
137177/137177 [==============================] - 17s 124us/step - loss: 7.7482 - accuracy: 0.0391 - val_loss: 6.8966 - val_accuracy: 0.0543
Epoch 2/50
137177/137177 [==============================] - 15s 107us/step - loss: 6.8565 - accuracy: 0.0509 - val_loss: 6.8254 - val_accuracy: 0.0543
Epoch 3/50
137177/137177 [==============================] - 15s 107us/step - loss: 6.7516 - accuracy: 0.0510 - val_loss: 6.7993 - val_accuracy: 0.0726
Epoch 4/50
137177/137177 [==============================] - 15s 107us/step - loss: 6.7029 - accuracy: 0.0564 - val_loss: 6.7925 - val_accuracy: 0.0727
Epoch 5/50
137177/137177 [==============================] - 15s 108us/step - loss: 6.6408 - accuracy: 0.0667 - val_loss: 6.7274 - val_accuracy: 0.0765
Epoch 6/50
137177/137177 [==============================] - 15s 108us/step - loss: 6.5507 - accuracy: 0.0738 - val_loss: 6.6666 - val_accuracy: 0.0797
Epoch 7/50
137177/137177 [=================

#### Unidirectional, 1 layer, trainable, lstm-dropout-0.2, lstm-recurrent-dropout-0.2

In [13]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False,
    lstm_dropout=0.2,
    lstm_recurrent_dropout=0.2,
)

In [14]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq-lstm_drop_0.2-lstm_rec_drop_0.2',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 137177 samples, validate on 58791 samples
Epoch 1/50
137177/137177 [==============================] - 14s 103us/step - loss: 7.7788 - accuracy: 0.0371 - val_loss: 6.8971 - val_accuracy: 0.0543
Epoch 2/50
137177/137177 [==============================] - 14s 102us/step - loss: 6.8567 - accuracy: 0.0505 - val_loss: 6.8270 - val_accuracy: 0.0543
Epoch 3/50
137177/137177 [==============================] - 14s 102us/step - loss: 6.7525 - accuracy: 0.0514 - val_loss: 6.7986 - val_accuracy: 0.0543
Epoch 4/50
137177/137177 [==============================] - 14s 100us/step - loss: 6.7025 - accuracy: 0.0573 - val_loss: 6.7902 - val_accuracy: 0.0683
Epoch 5/50
137177/137177 [==============================] - 14s 101us/step - loss: 6.6399 - accuracy: 0.0682 - val_loss: 6.7259 - val_accuracy: 0.0769
Epoch 6/50
137177/137177 [==============================] - 14s 101us/step - loss: 6.5436 - accuracy: 0.0750 - val_loss: 6.6538 - val_accuracy: 0.0804
Epoch 7/50
137177/137177 [=================

#### Unidirectional, 1 layer, trainable, lstm-dropout-0.5, lstm-recurrent-dropout-0.5

In [15]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False,
    lstm_dropout=0.5,
    lstm_recurrent_dropout=0.5,
)

In [16]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq-lstm_drop_0.5-lstm_rec_drop_0.5',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 137177 samples, validate on 58791 samples
Epoch 1/50
137177/137177 [==============================] - 15s 107us/step - loss: 7.7921 - accuracy: 0.0378 - val_loss: 6.8946 - val_accuracy: 0.0543
Epoch 2/50
137177/137177 [==============================] - 15s 108us/step - loss: 6.8547 - accuracy: 0.0505 - val_loss: 6.8311 - val_accuracy: 0.0543
Epoch 3/50
137177/137177 [==============================] - 15s 108us/step - loss: 6.7629 - accuracy: 0.0507 - val_loss: 6.8057 - val_accuracy: 0.0543
Epoch 4/50
137177/137177 [==============================] - 15s 107us/step - loss: 6.7194 - accuracy: 0.0527 - val_loss: 6.8063 - val_accuracy: 0.0543
Epoch 5/50
137177/137177 [==============================] - 15s 107us/step - loss: 6.6749 - accuracy: 0.0633 - val_loss: 6.7488 - val_accuracy: 0.0732
Epoch 6/50
137177/137177 [==============================] - 15s 107us/step - loss: 6.5788 - accuracy: 0.0726 - val_loss: 6.6579 - val_accuracy: 0.0780
Epoch 7/50
137177/137177 [=================

## Compare Model

<tr>
    <th>Name of Model</th>
    <th>LTSM Layers</th>
    <th>Bidirectional</th>
    <th>Trainable Embeddings</th>
    <th>Sequence Length</th>
    <th>LSTM Dropout</th>
    <th>LSTM Recurrent Dropout</th>
    <th>Val Loss</th>
    <th>Val Accuracy</th>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.5235</td>
    <td>0.1715</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.4229</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.2247</td>
    <td>0.1638</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.4449</td>
    <td>0.1725</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.2899</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.1667</td>
    <td>0.1719</td>
</tr>

# Sample Output

## Load Model

In [7]:
from keras.models import load_model
# Get Model Weights and Architecture

# MODELS_DIR = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
MODELS_DIR = '/content/drive/My Drive/Code/autocomplete_me/models'
model_filepath = os.path.join(MODELS_DIR, f'{content_type}_uni-1_layer-trainable-50_seq.h5')

model = load_model(model_filepath)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [8]:
# Get Text Data
# text = reader.read_bbc_politics()

TRAINING_LENGTH = 50
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=TRAINING_LENGTH)

There are 14136 unique words.
There are 183458 sequences.


In [9]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=50,
    diversity=1,
    n_gen=1
)

In [10]:
' '.join(word for word in original_sequence)

'He has also started to recruit 25,000 Community Support Officers (CSOs) and the new plans would allow all police forces to give them the power to detain suspects . An extra £50m was promised on Tuesday so 2,000 new CSOs can be recruited now rather than next year . Within'

In [11]:
' '.join(word for word in gen_list[0])

'< --- > mass squabbling in them living and get in ducking by nationality to door , leaving to tackle clear that you know him as the Poland . But he was wrong up their backbencher plans , which chairman for responsibility for new benefits to an £71,433 had confess of his Union'

In [12]:
' '.join(word for word in a)

"< --- > two years , every force will be expected to keep to a coppers' contract on what kind of service the public can expect . A Mori poll this summer suggested policing , unlike health and education , was the one major public service where people were less satisfied the more"

In [31]:
sentence = 'Stocks of major large technology firms are becoming'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 5, 479, 714, 2354, 2532, 25, 1215]
role . Many accused the US . Mr Campbell said the cost of Constitutional second record - in tax and Amnesty intervals . And it can new pair around as the English country of or match the House of other year's binge child , UKIP Party feels Veritas , .


In [15]:
sentence = 'However, there have been many instances of'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 55, 21, 42, 169, 5319, 5]
its party's approach to the watchdog AND visit to part for a maximum credits have have shown to new slow at Iraq has straightforward by they , gaming minister has under affect Washington would not Defra's trying on the next Dem 10 vote , despite act in it's changed had
